In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [26]:
df = pd.read_csv('Data/Finance_Exam_Data.csv')
df['Date'] = pd.to_datetime(df['Date'])

In [27]:
df.head()

,Date,Vodacom,Anglo American,KUMBA,ABSA,Woolworths,ALSE (benchmark),Risk- Free-Rate
0,2007-01-03,28.797146,2805.500000,11899.0,12700.0,23.501499,4.817180,0.185
1,2007-01-04,28.786953,2701.100098,11399.0,12600.0,23.214399,4.715587,0.190
2,2007-01-05,29.510704,2637.360107,11300.0,12286.0,23.006500,4.774849,0.180
3,2007-01-08,28.725790,2560.439941,11000.0,12370.0,23.214399,4.825645,0.175
4,2007-01-09,29.194698,2602.199951,11090.0,12170.0,22.580799,4.808713,NaN


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3353 entries, 0 to 3352
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              3353 non-null   datetime64[ns]
 1   Vodacom           3353 non-null   float64       
 2   Anglo American    3353 non-null   float64       
 3   KUMBA             3353 non-null   float64       
 4   ABSA              3353 non-null   float64       
 5   Woolworths        3353 non-null   float64       
 6   ALSE (benchmark)  3353 non-null   float64       
 7   Risk- Free-Rate   2753 non-null   float64       
dtypes: datetime64[ns](1), float64(7)
memory usage: 209.7 KB


In [29]:
stocks = ['Vodacom','Anglo American','KUMBA','ABSA','Woolworths']

In [34]:
for stock in stocks:
    df[f'{stock} Return'] = df[stock].pct_change(periods=1)

In [35]:
df.head()

,Date,Vodacom,Anglo American,KUMBA,ABSA,Woolworths,ALSE (benchmark),Risk- Free-Rate,Vodacom Return,Anglo American Return,KUMBA Return,ABSA Return,Woolworths Return
0,2007-01-03,28.797146,2805.500000,11899.0,12700.0,23.501499,4.817180,0.185,NaN,NaN,NaN,NaN,NaN
1,2007-01-04,28.786953,2701.100098,11399.0,12600.0,23.214399,4.715587,0.190,-0.000354,-0.037213,-0.042020,-0.007874,-0.012216
2,2007-01-05,29.510704,2637.360107,11300.0,12286.0,23.006500,4.774849,0.180,0.025142,-0.023598,-0.008685,-0.024921,-0.008956
3,2007-01-08,28.725790,2560.439941,11000.0,12370.0,23.214399,4.825645,0.175,-0.026598,-0.029166,-0.026549,0.006837,0.009037
4,2007-01-09,29.194698,2602.199951,11090.0,12170.0,22.580799,4.808713,NaN,0.016324,0.016310,0.008182,-0.016168,-0.027293


In [36]:
df['Excess Market Return'] = df['ALSE (benchmark)'] - df['Risk- Free-Rate ']

In [37]:
df.head()

,Date,Vodacom,Anglo American,KUMBA,ABSA,Woolworths,ALSE (benchmark),Risk- Free-Rate,Vodacom Return,Anglo American Return,KUMBA Return,ABSA Return,Woolworths Return,Excess Market Return
0,2007-01-03,28.797146,2805.500000,11899.0,12700.0,23.501499,4.817180,0.185,NaN,NaN,NaN,NaN,NaN,4.632180
1,2007-01-04,28.786953,2701.100098,11399.0,12600.0,23.214399,4.715587,0.190,-0.000354,-0.037213,-0.042020,-0.007874,-0.012216,4.525587
2,2007-01-05,29.510704,2637.360107,11300.0,12286.0,23.006500,4.774849,0.180,0.025142,-0.023598,-0.008685,-0.024921,-0.008956,4.594849
3,2007-01-08,28.725790,2560.439941,11000.0,12370.0,23.214399,4.825645,0.175,-0.026598,-0.029166,-0.026549,0.006837,0.009037,4.650645
4,2007-01-09,29.194698,2602.199951,11090.0,12170.0,22.580799,4.808713,NaN,0.016324,0.016310,0.008182,-0.016168,-0.027293,NaN


In [39]:
df['Excess Market Return'].iloc[0] = np.nan

C:\Users\clint\AppData\Local\Temp\ipykernel_33248\3425510600.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Excess Market Return'].iloc[0] = np.nan


In [41]:
df.head()

,Date,Vodacom,Anglo American,KUMBA,ABSA,Woolworths,ALSE (benchmark),Risk- Free-Rate,Vodacom Return,Anglo American Return,KUMBA Return,ABSA Return,Woolworths Return,Excess Market Return
0,2007-01-03,28.797146,2805.500000,11899.0,12700.0,23.501499,4.817180,0.185,NaN,NaN,NaN,NaN,NaN,NaN
1,2007-01-04,28.786953,2701.100098,11399.0,12600.0,23.214399,4.715587,0.190,-0.000354,-0.037213,-0.042020,-0.007874,-0.012216,4.525587
2,2007-01-05,29.510704,2637.360107,11300.0,12286.0,23.006500,4.774849,0.180,0.025142,-0.023598,-0.008685,-0.024921,-0.008956,4.594849
3,2007-01-08,28.725790,2560.439941,11000.0,12370.0,23.214399,4.825645,0.175,-0.026598,-0.029166,-0.026549,0.006837,0.009037,4.650645
4,2007-01-09,29.194698,2602.199951,11090.0,12170.0,22.580799,4.808713,NaN,0.016324,0.016310,0.008182,-0.016168,-0.027293,NaN


In [45]:
expected_returns = {
    'Vodacom':df['Vodacom Return'].mean(),
    'Anglo American':df['Anglo American Return'].mean(),
    'KUMBA':df['KUMBA Return'].mean(),
    'ABSA':df['ABSA Return'].mean(),
    'Woolworths':df['Woolworths Return'].mean()
}

In [47]:
expected_returns_df = pd.DataFrame(expected_returns,index=['Expected Returns'])

In [48]:
expected_returns_df

,Vodacom,Anglo American,KUMBA,ABSA,Woolworths
Expected Returns,0.000067,0.023503,0.000797,0.000205,0.00023
